<a href="https://colab.research.google.com/github/PedroRibeiroM/aulaJoao/blob/main/ValidacaoBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import threading
import time
import random

class Block:
    def __init__(self, data):
        self.data = data
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        # Criar um hash a partir do conteúdo do bloco
        return hashlib.sha256(self.data.encode('utf-8')).hexdigest()

    def update_data(self, new_data):
        self.data = new_data
        self.hash = self.calculate_hash()

def block_validator(block, original_block, stop_event):
    """
    Thread que valida o hash do bloco periodicamente.
    Se o hash atual for diferente do original, o bloco é considerado alterado e será descartado.
    """
    print("Iniciando validação de bloco...")
    while not stop_event.is_set():
        current_hash = block.hash
        original_hash = original_block.hash

        if current_hash != original_hash:
            print(f"Alteração detectada! Hash atual: {current_hash} != Hash original: {original_hash}")
            print("Bloco foi modificado! Sinalizando descarte.")
            stop_event.set()  # Sinaliza para parar a execução
        else:
            print(f"Bloco válido! Hash atual: {current_hash}")

        time.sleep(2)  # Checa a cada 2 segundos

def modify_block_periodically(block, stop_event):
    """
    Simula modificações no bloco após intervalos de tempo aleatórios.
    """
    while not stop_event.is_set():
        # Aguarda um tempo aleatório entre 3 e 8 segundos antes de modificar o bloco
        time_to_wait = random.randint(3, 8)
        time.sleep(time_to_wait)

        # Modifica o bloco com dados aleatórios
        new_data = f"Nova transação {random.randint(1000, 9999)}"
        print(f"Modificando o bloco com novos dados: {new_data}")
        block.update_data(new_data)

def main():
    # Dados originais do bloco
    original_data = "Transação inicial 1234"
    bloco_original = Block(original_data)
    bloco_atual = Block(original_data)  # Cópia para modificações

    stop_event = threading.Event()

    # Cria e inicia a thread de validação
    validator_thread = threading.Thread(target=block_validator, args=(bloco_atual, bloco_original, stop_event))
    validator_thread.start()

    # Cria e inicia a thread que modifica o bloco periodicamente
    modifier_thread = threading.Thread(target=modify_block_periodically, args=(bloco_atual, stop_event))
    modifier_thread.start()

    # Mantém a execução principal por um tempo, depois encerra
    try:
        while not stop_event.is_set():
            time.sleep(1)
    except KeyboardInterrupt:
        stop_event.set()
        print("Finalizando execução...")

    # Espera as threads terminarem
    validator_thread.join()
    modifier_thread.join()

if __name__ == "__main__":
    main()


Iniciando validação de bloco...
Bloco válido! Hash atual: 47dbfe9aae34603754946876f69433c132600abb2f04d3b564cfd75f214e08d5
Bloco válido! Hash atual: 47dbfe9aae34603754946876f69433c132600abb2f04d3b564cfd75f214e08d5
Bloco válido! Hash atual: 47dbfe9aae34603754946876f69433c132600abb2f04d3b564cfd75f214e08d5
Modificando o bloco com novos dados: Nova transação 4248
Alteração detectada! Hash atual: 4a3d3e23fa70ee6f55926414c10ac6b43284b605d1f2472d0c5b79fa2f2356a6 != Hash original: 47dbfe9aae34603754946876f69433c132600abb2f04d3b564cfd75f214e08d5
Bloco foi modificado! Sinalizando descarte.
Modificando o bloco com novos dados: Nova transação 6015
